# BDA ANALYSEPROJEKT MIETPREISE DEUTSCHLAND 2023


Imports

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import seaborn as sns
from scipy.stats import lognorm, kstest, norm, shapiro

%matplotlib inline

Einlesen der Exel Datei und herausfiltern der Daten die wir wollen


In [4]:
KRS1222 = df = pd.read_excel("Deutschlandatlas-Daten.xlsx", sheet_name="Deutschlandatlas_KRS1222")
KRS1222.head(5)

,KRS1222,Kreisname,fl_suv,fl_landw,fl_wald,bev_binw,bev_ausw,ko_kasskred,bev_u18,bev_18_65,...,auto,v_harzt,v_karzt,schule_oabschl,kbetr_u3,kbetr_ue3,kbetr_ue6,kinder_bg,straft,einbr
0,1001000,"Flensburg, Stadt",53.50,22.12,6.73,49.81,167.15,0.00,16.02,64.02,...,495.3,65.1,66.4,8.09,38.41,86.78,26.74,21.55,10767.152890,117.774176
1,1002000,"Kiel, Landeshauptstadt",58.78,23.45,6.94,-12.80,115.33,181.66,15.03,65.99,...,449.8,67.2,78.4,9.34,36.44,91.37,22.46,24.44,10684.773350,117.069075
2,1003000,"Lübeck, Hansestadt",37.86,30.23,15.44,30.67,116.23,268.90,15.22,61.36,...,467.0,67.5,83.8,11.34,39.60,93.48,14.27,20.98,10322.107339,131.135514
3,1004000,"Neumünster, Stadt",51.81,37.72,4.65,-286.79,359.36,0.00,16.45,60.50,...,543.5,58.2,65.1,9.51,31.59,83.00,20.39,20.73,18117.783200,225.151569
4,1051000,Dithmarschen,11.28,75.22,3.83,66.47,105.51,0.00,15.74,58.73,...,621.3,63.0,45.7,12.14,25.69,87.67,12.46,14.75,5401.029190,116.079614
